# Top Spending Cohorts

In [27]:
import polars as pl
from datetime import datetime, timedelta
from sys import exit

# Assuming claims_df is a Polars DataFrame with the following schema:
# - cohort_id: identifier for the cohort (string)
# - cohort_name: human-readable cohort name (string)
# - claim_date: the date of the claim (date/datetime)
# - claim_amount: the dollar amount of the claim (float)


def compute_top_spending_cohorts(claims_df: pl.DataFrame, reference_date: str, number_of_rows: int = 3) -> pl.DataFrame:
    """
    Compute the top 3 outlier cohorts based on historical (up to last 24-months) spend.

    Parameters
    ----------
    claims_df : pl.DataFrame
        A DataFrame containing claims with columns: cohort_id, cohort_name, claim_date, claim_amount.
    reference_date : str
        A YYYY-MM-DD string indicating the month for which to compute outliers (e.g., '2025-04-30').
    number_of_rows : int
        The number of rows to return (default is 3).

    Returns
    -------
    pl.DataFrame
        A DataFrame with top 3 outlier cohorts sorted by spend, containing columns:
        cohort_id, cohort_name, spend
    """

    # Parse reference_date into datetime object
    reference_date = datetime.strptime(reference_date, "%Y-%m-%d")
    twenty_four_months_ago_from_reference_date = (reference_date - timedelta(days=365*2)).replace(day=1)

    # Create a LazyFrame for performance
    lf = claims_df.lazy()

    # 1) Extract historical (up to last 24-months) data
    previous_24_months_to_current_date = (
        lf
        .with_columns(
            pl.col("claim_date").cast(pl.Date).alias("date")
        )
        .filter(
            (pl.col("date") >= pl.lit(twenty_four_months_ago_from_reference_date)) &
            (pl.col("date") <= pl.lit(reference_date))
        )
        .group_by(pl.col("cohort_name"))
        .agg(
            pl.sum("claim_amount").alias("spend")
        )
        .sort(pl.col("spend"), descending=True, nulls_last=True)
        .limit(number_of_rows)
    )

    # Collect to materialize results
    return previous_24_months_to_current_date

In [28]:
from cohorts_diseases import get_cohort_diseases_icd_level_1, get_cohort_diseases_icd_level_2, get_cohort_diseases_chronic, get_cohort_diseases_trigger_level_2

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    print(f"Cohort Diseases ICD Level 1: {compute_top_spending_cohorts(get_cohort_diseases_icd_level_1('113463').collect(), '2025-04-30', number_of_rows=10).collect()}")
    print(f"Cohort Diseases ICD Level 2: {compute_top_spending_cohorts(get_cohort_diseases_icd_level_2('113463').collect(), '2025-04-30', number_of_rows=10).collect()}")
    print(f"Cohort Diseases Chronic: {compute_top_spending_cohorts(get_cohort_diseases_chronic('113463').collect(), '2025-04-30', number_of_rows=10).collect()}")
    print(f"Cohort Diseases Trigger Level 2: {compute_top_spending_cohorts(get_cohort_diseases_trigger_level_2('113463').collect(), '2025-04-30', number_of_rows=10).collect()}")


Cohort Diseases ICD Level 1: shape: (10, 2)
┌─────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│ cohort_name                                                                             ┆     spend │
│ ---                                                                                     ┆       --- │
│ str                                                                                     ┆       f64 │
╞═════════════════════════════════════════════════════════════════════════════════════════╪═══════════╡
│ Factors influencing health status and contact with health services                      ┆ 645028.95 │
│ Diseases of the musculoskeletal system and connective tissue                            ┆ 622672.93 │
│ Neoplasms                                                                               ┆ 550239.91 │
│ Diseases of the genitourinary system                                                    ┆ 534873.37 │
│ Diseases of the ci

In [29]:
from cohorts_procedures import get_cohort_procedures

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    print(f"Cohort Procedures: {compute_top_spending_cohorts(get_cohort_procedures('113463').collect(), '2025-04-30', number_of_rows=10).collect()}")


Cohort Procedures: shape: (10, 2)
┌──────────────────────────────────────────────────┬────────────┐
│ cohort_name                                      ┆      spend │
│ ---                                              ┆        --- │
│ str                                              ┆        f64 │
╞══════════════════════════════════════════════════╪════════════╡
│ Medical Care                                     ┆ 2548810.11 │
│ *                                                ┆ 1590013.29 │
│ Surgery                                          ┆  244787.15 │
│ Diagnostic Laboratory                            ┆  239978.81 │
│ Medical Care | Whole Blood                       ┆  202720.00 │
│ Anesthesia                                       ┆  164686.81 │
│ Lump Sum Purchase of DME, Prosthetics, Orthotics ┆   95967.71 │
│ Diagnostic Radiology                             ┆   85409.32 │
│ Outpatient Mental Health Treatment Limitation    ┆   59178.31 │
│ Other Medical Items or Services         

In [30]:
from cohorts_facilities import get_cohort_facilities
pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    print(f"Cohort Facilities: {compute_top_spending_cohorts(get_cohort_facilities('113463').collect(), '2025-04-30', number_of_rows=10).collect()}")

Cohort Facilities: shape: (10, 2)
┌───────────────────────────────┬────────────┐
│ cohort_name                   ┆      spend │
│ ---                           ┆        --- │
│ str                           ┆        f64 │
╞═══════════════════════════════╪════════════╡
│ Inpatient Hospital            ┆ 1431891.62 │
│ Office                        ┆  928983.50 │
│ Emergency Room  Hospital      ┆  917308.47 │
│ Ambulatory Surgical Center    ┆  876069.91 │
│ On Campus-Outpatient Hospital ┆  840563.80 │
│ Independent Laboratory        ┆  142807.53 │
│ Home                          ┆  114979.92 │
│ Telehealth                    ┆   25647.33 │
│ Skilled Nursing Facility      ┆   20567.42 │
│ Unassigned                    ┆   17763.78 │
└───────────────────────────────┴────────────┘


In [31]:
from cohorts_drugs import get_cohort_drugs_usage
pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    print(f"Cohort Facilities: {compute_top_spending_cohorts(get_cohort_drugs_usage('113463').collect(), '2025-04-30', number_of_rows=10).collect()}")

Cohort Facilities: shape: (10, 2)
┌─────────────────────────────────────────┬───────────┐
│ cohort_name                             ┆     spend │
│ ---                                     ┆       --- │
│ str                                     ┆       f64 │
╞═════════════════════════════════════════╪═══════════╡
│ Immunosuppressants                      ┆ 874837.74 │
│ Drugs Used In Diabetes                  ┆ 496343.81 │
│ Antihemorrhagics                        ┆ 260377.88 │
│ Psycholeptics                           ┆  74805.97 │
│ Analgesics                              ┆  73975.05 │
│ Antithrombotic Agents                   ┆  63408.35 │
│ Other Nervous System Drugs              ┆  56753.77 │
│ Antineoplastic AgentsImmunosuppressants ┆  41357.14 │
│ Drugs For Obstructive Airway Diseases   ┆  39069.75 │
│ Psychoanaleptics                        ┆  37507.89 │
└─────────────────────────────────────────┴───────────┘


# Surge in Spending Cohorts

In [32]:
import polars as pl
from datetime import datetime, timedelta
from sys import exit
# Assuming claims_df is a Polars DataFrame with the following schema:
# - cohort_id: identifier for the cohort (string)
# - cohort_name: human-readable cohort name (string)
# - claim_date: the date of the claim (date/datetime)
# - claim_amount: the dollar amount of the claim (float)


def compute_outlier_cohorts(claims_df: pl.DataFrame, reference_date: str, number_of_rows: int = 3, include_infinity_percentage: bool = False) -> pl.DataFrame:
    """
    Compute the top 3 outlier cohorts based on current month spend vs. last 24-month average spend.

    Parameters
    ----------
    claims_df : pl.DataFrame
        A DataFrame containing claims with columns: cohort_id, cohort_name, claim_date, claim_amount.
    reference_date : str
        A YYYY-MM-DD string indicating the month for which to compute outliers (e.g., '2025-04-30').
    number_of_rows : int
        The number of rows to return (default is 3).
    include_infinity_percentage : bool
        Whether to include cohorts with infinite percentage increase, observed with the average of historical spend is zero(default is False).

    Returns
    -------
    pl.DataFrame
        A DataFrame with top 3 outlier cohorts sorted by percent increase, containing columns:
        cohort_id, cohort_name, current_month_spend, avg_24m_spend, pct_increase
    """
    
    # Parse reference_date into datetime object
    reference_date = datetime.strptime(reference_date, "%Y-%m-%d")
    last_30_days_from_reference_date = (reference_date - timedelta(days=30))
    twenty_four_months_ago_from_reference_date = (reference_date - timedelta(days=365*2)).replace(day=1)

    # Create a LazyFrame for performance
    lf = claims_df.lazy()

    # 1) Extract current month data
    current_month = (
        lf
        .with_columns(
            pl.col("claim_date").cast(pl.Date).alias("date")
        )
        .filter(
            (pl.col("date") >= pl.lit(last_30_days_from_reference_date)) &
            (pl.col("date") <= pl.lit(reference_date))
        )
        .group_by(pl.col("cohort_name"))
        .agg(
            pl.sum("claim_amount").alias("current_month_spend")
        )
        .filter(pl.col("current_month_spend") > 0)
    )

    # 2) Compute 24-month window prior to reference month
    historical_24m = (
        lf
        .with_columns(
            pl.col("claim_date").cast(pl.Date).alias("date")
        )
        .filter(
            (pl.col("date") >= pl.lit(twenty_four_months_ago_from_reference_date)) &
            (pl.col("date") <= pl.lit(reference_date))
        )
        .group_by(pl.col("cohort_name"))
        .agg(
            (pl.sum("claim_amount") / 24).alias("avg_24m_spend")
        )
        .with_columns(pl.col("avg_24m_spend").fill_null(0.0).fill_nan(0.0))
    )

    # 3) Join current month and historical averages
    if include_infinity_percentage:
        outlier_df = (
            current_month
            .join(historical_24m, on=["cohort_name"], how="left")
            .with_columns(
                pl.when(pl.col("avg_24m_spend") != 0)
                .then(((pl.col("current_month_spend") - pl.col("avg_24m_spend")) / 
                    pl.col("avg_24m_spend")) * 100)
                .otherwise(float("inf"))
                .alias("pct_increase")
            )
            .sort(["pct_increase"], descending=True, nulls_last=True)
            .limit(number_of_rows)
        )
    else:
        outlier_df = (
            current_month
            .join(historical_24m, on=["cohort_name"], how="left")
            .with_columns(
                pl.when(pl.col("avg_24m_spend") != 0)
                .then(((pl.col("current_month_spend") - pl.col("avg_24m_spend")) / 
                    pl.col("avg_24m_spend")) * 100)
                .otherwise(float("inf"))
                .alias("pct_increase")
            )
            .filter(pl.col("pct_increase") != float("inf"))
            .sort(["pct_increase"], descending=True, nulls_last=True)
            .limit(number_of_rows)
        )

    # Collect to materialize results
    return outlier_df

In [33]:
from cohorts_diseases import get_cohort_diseases_icd_level_1, get_cohort_diseases_icd_level_2, get_cohort_diseases_chronic, get_cohort_diseases_trigger_level_2

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Diseases ICD Level 1: {compute_outlier_cohorts(get_cohort_diseases_icd_level_1('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Diseases ICD Level 1: {compute_outlier_cohorts(get_cohort_diseases_icd_level_1('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Diseases ICD Level 2: {compute_outlier_cohorts(get_cohort_diseases_icd_level_2('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Diseases ICD Level 2: {compute_outlier_cohorts(get_cohort_diseases_icd_level_2('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Diseases Chronic: {compute_outlier_cohorts(get_cohort_diseases_chronic('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Diseases Chronic: {compute_outlier_cohorts(get_cohort_diseases_chronic('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Diseases Trigger Level 2: {compute_outlier_cohorts(get_cohort_diseases_trigger_level_2('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Diseases Trigger Level 2: {compute_outlier_cohorts(get_cohort_diseases_trigger_level_2('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")


Cohort Diseases ICD Level 1: shape: (10, 4)
┌─────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────┬───────────────┬──────────────┐
│ cohort_name                                                                             ┆ current_month_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                                                                     ┆                 --- ┆           --- ┆          --- │
│ str                                                                                     ┆                 f64 ┆           f64 ┆          f64 │
╞═════════════════════════════════════════════════════════════════════════════════════════╪═════════════════════╪═══════════════╪══════════════╡
│ Pregnancy, childbirth and the puerperium                                                ┆            15701.51 ┆       6757.30 ┆       132.36 │
│ Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classif

In [34]:
from cohorts_procedures import get_cohort_procedures

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Procedures: {compute_outlier_cohorts(get_cohort_procedures('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Procedures: {compute_outlier_cohorts(get_cohort_procedures('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Procedures: shape: (10, 4)
┌────────────────────────────────────────────────────────┬─────────────────────┬───────────────┬──────────────┐
│ cohort_name                                            ┆ current_month_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                                    ┆                 --- ┆           --- ┆          --- │
│ str                                                    ┆                 f64 ┆           f64 ┆          f64 │
╞════════════════════════════════════════════════════════╪═════════════════════╪═══════════════╪══════════════╡
│ Medical Care | Whole Blood                             ┆            71574.32 ┆       8446.67 ┆       747.37 │
│ Vision Items or Services                               ┆              282.23 ┆        164.81 ┆        71.25 │
│ Lump Sum Purchase of DME, Prosthetics, Orthotics       ┆             6689.37 ┆       3998.65 ┆        67.29 │
│ Rental of DME                                          ┆            

In [35]:
from cohorts_facilities import get_cohort_facilities
pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_facilities('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_facilities('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Facilities: shape: (10, 4)
┌────────────────────────────────┬─────────────────────┬───────────────┬──────────────┐
│ cohort_name                    ┆ current_month_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                            ┆                 --- ┆           --- ┆          --- │
│ str                            ┆                 f64 ┆           f64 ┆          f64 │
╞════════════════════════════════╪═════════════════════╪═══════════════╪══════════════╡
│ Unassigned                     ┆             1873.22 ┆        740.16 ┆       153.08 │
│ Rural Health Clinic            ┆              321.00 ┆        133.43 ┆       140.57 │
│ Urgent Care Facility           ┆             1298.64 ┆        717.88 ┆        80.90 │
│ Office                         ┆            37214.75 ┆      38707.65 ┆        -3.86 │
│ Independent Laboratory         ┆             5017.37 ┆       5950.31 ┆       -15.68 │
│ Telehealth                     ┆              894.41 ┆       1068.64 ┆       -16.30 

In [36]:
from cohorts_drugs import get_cohort_drugs_usage
pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_drugs_usage('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_drugs_usage('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Facilities: shape: (10, 4)
┌───────────────────────────────────────────────────────────────────────┬─────────────────────┬───────────────┬──────────────┐
│ cohort_name                                                           ┆ current_month_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                                                   ┆                 --- ┆           --- ┆          --- │
│ str                                                                   ┆                 f64 ┆           f64 ┆          f64 │
╞═══════════════════════════════════════════════════════════════════════╪═════════════════════╪═══════════════╪══════════════╡
│ VasoprotectivesAntipruritics, Incl. Antihistamines, Anesthetics, Etc. ┆               98.80 ┆         16.45 ┆       500.46 │
│ Antiobesity Preparations, Excl. Diet Products                         ┆               22.91 ┆          4.01 ┆       471.98 │
│ Diuretics                                                             ┆    

In [37]:
from cohorts_demographics import get_cohort_demographics_ages, get_cohort_demographics_relationships, get_cohort_demographics_genders

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_demographics_ages('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_demographics_ages('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_demographics_genders('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_demographics_genders('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_demographics_relationships('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_demographics_relationships('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Facilities: shape: (5, 4)
┌─────────────┬─────────────────────┬───────────────┬──────────────┐
│ cohort_name ┆ current_month_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---         ┆                 --- ┆           --- ┆          --- │
│ str         ┆                 f64 ┆           f64 ┆          f64 │
╞═════════════╪═════════════════════╪═══════════════╪══════════════╡
│ 27-40       ┆            38187.31 ┆      33986.09 ┆        12.36 │
│ 65+         ┆             4735.67 ┆       9620.86 ┆       -50.78 │
│ 0-18        ┆             8685.67 ┆      18078.07 ┆       -51.95 │
│ 41-64       ┆            73806.81 ┆     155195.28 ┆       -52.44 │
│ 19-26       ┆             1948.11 ┆       7978.74 ┆       -75.58 │
└─────────────┴─────────────────────┴───────────────┴──────────────┘
Cohort Facilities: shape: (2, 4)
┌─────────────┬─────────────────────┬───────────────┬──────────────┐
│ cohort_name ┆ current_month_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---         ┆                 --- ┆

In [38]:
from cohorts_providers import get_cohort_individual_medical_provider, get_cohort_medical_provider_speciality, get_cohort_individual_rx_provider

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_individual_medical_provider('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_individual_medical_provider('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_individual_rx_provider('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_individual_rx_provider('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_medical_provider_speciality('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_outlier_cohorts(get_cohort_medical_provider_speciality('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Facilities: shape: (10, 4)
┌───────────────────────────────────────────┬─────────────────────┬───────────────┬──────────────┐
│ cohort_name                               ┆ current_month_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                       ┆                 --- ┆           --- ┆          --- │
│ str                                       ┆                 f64 ┆           f64 ┆          f64 │
╞═══════════════════════════════════════════╪═════════════════════╪═══════════════╪══════════════╡
│ NILESH NANGRANI                           ┆              191.59 ┆          7.98 ┆      2300.00 │
│ DAVID RICE                                ┆               15.57 ┆          0.65 ┆      2300.00 │
│ BROOKE HARNISCH                           ┆              113.42 ┆          4.73 ┆      2300.00 │
│ REBECCA KING                              ┆              100.00 ┆          4.17 ┆      2300.00 │
│ TUSCAN SURGERY CENTER AT LAS COLINAS  LLC ┆              649.99 ┆        

# Emerging Spending Categories

In [39]:
import polars as pl
from datetime import datetime, timedelta
from sys import exit
# Assuming claims_df is a Polars DataFrame with the following schema:
# - cohort_id: identifier for the cohort (string)
# - cohort_name: human-readable cohort name (string)
# - claim_date: the date of the claim (date/datetime)
# - claim_amount: the dollar amount of the claim (float)


def compute_emerging_cohorts(claims_df: pl.DataFrame, reference_date: str, number_of_rows: int = 3, include_infinity_percentage: bool = False) -> pl.DataFrame:
    """
    Compute the top 3 outlier cohorts based on last 3 months average spend vs. last 24 months average spend.

    Parameters
    ----------
    claims_df : pl.DataFrame
        A DataFrame containing claims with columns: cohort_id, cohort_name, claim_date, claim_amount.
    reference_date : str
        A YYYY-MM-DD string indicating the month for which to compute outliers (e.g., '2025-04-30').
    number_of_rows : int
        The number of rows to return (default is 3).
    include_infinity_percentage : bool
        Whether to include cohorts with infinite percentage increase, observed with the average of historical spend is zero(default is False).

    Returns
    -------
    pl.DataFrame
        A DataFrame with top 3 outlier cohorts sorted by percent increase, containing columns:
        cohort_id, cohort_name, last_3m_avg_spend, historical_24m, pct_increase
    """

    # Parse reference_date into datetime object
    reference_date = datetime.strptime(reference_date, "%Y-%m-%d")
    last_90_days_from_reference_date = (reference_date - timedelta(days=90))
    twenty_four_months_ago_from_reference_date = (reference_date - timedelta(days=365*2)).replace(day=1)

    # Create a LazyFrame for performance
    lf = claims_df.lazy()

    # 1) Extract last 3 months average data
    last_3m_avg_spend = (
        lf
        .with_columns(
            pl.col("claim_date").cast(pl.Date).alias("date")
        )
        .filter(
            (pl.col("date") >= pl.lit(last_90_days_from_reference_date)) &
            (pl.col("date") <= pl.lit(reference_date))
        )
        .group_by(pl.col("cohort_name"))
        .agg(
            (pl.sum("claim_amount") / 3).alias("last_3m_avg_spend")
        )
        .with_columns(pl.col("last_3m_avg_spend").fill_null(0.0).fill_nan(0.0))
        .filter(pl.col("last_3m_avg_spend") > 0)
    )

    # 2) Compute 24-month window prior to reference month
    historical_24m = (
        lf
        .with_columns(
            pl.col("claim_date").cast(pl.Date).alias("date")
        )
        .filter(
            (pl.col("date") >= pl.lit(twenty_four_months_ago_from_reference_date)) &
            (pl.col("date") < pl.lit(reference_date))
        )
        .group_by(pl.col("cohort_name"))
        .agg(
            (pl.sum("claim_amount") / 24).alias("avg_24m_spend")
        )
        .with_columns(pl.col("avg_24m_spend").fill_null(0.0).fill_nan(0.0))
    )

    # 3) Join last 3 months average and historical averages
    if include_infinity_percentage:
        outlier_df = (
        last_3m_avg_spend
        .join(historical_24m, on=["cohort_name"], how="left")
        .with_columns(
            pl.when(pl.col("avg_24m_spend") != 0)
            .then(((pl.col("last_3m_avg_spend") - pl.col("avg_24m_spend")) / 
                pl.col("avg_24m_spend")) * 100)
            .otherwise(float("inf"))
            .alias("pct_increase")
        )
        .sort(["pct_increase", "last_3m_avg_spend"], descending=True, nulls_last=True)
        .limit(number_of_rows)
    )
    else:
        outlier_df = (
        last_3m_avg_spend
        .join(historical_24m, on=["cohort_name"], how="left")
        .with_columns(
            pl.when(pl.col("avg_24m_spend") != 0)
            .then(((pl.col("last_3m_avg_spend") - pl.col("avg_24m_spend")) / 
                pl.col("avg_24m_spend")) * 100)
            .otherwise(float("inf"))
            .alias("pct_increase")
        )
        .filter(pl.col("pct_increase") != float("inf"))
        .sort(["pct_increase", "last_3m_avg_spend"], descending=True, nulls_last=True)
        .limit(number_of_rows)
    )
    

    # Collect to materialize results
    return outlier_df

In [40]:
from cohorts_diseases import get_cohort_diseases_icd_level_1, get_cohort_diseases_icd_level_2, get_cohort_diseases_chronic, get_cohort_diseases_trigger_level_2

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Diseases ICD Level 1: {compute_emerging_cohorts(get_cohort_diseases_icd_level_1('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Diseases ICD Level 1: {compute_emerging_cohorts(get_cohort_diseases_icd_level_1('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Diseases ICD Level 2: {compute_emerging_cohorts(get_cohort_diseases_icd_level_2('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Diseases ICD Level 2: {compute_emerging_cohorts(get_cohort_diseases_icd_level_2('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Diseases Chronic: {compute_emerging_cohorts(get_cohort_diseases_chronic('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Diseases Chronic: {compute_emerging_cohorts(get_cohort_diseases_chronic('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Diseases Trigger Level 2: {compute_emerging_cohorts(get_cohort_diseases_trigger_level_2('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Diseases Trigger Level 2: {compute_emerging_cohorts(get_cohort_diseases_trigger_level_2('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")


Cohort Diseases ICD Level 1: shape: (10, 4)
┌─────────────────────────────────────────────────────────────────────────────────────────┬───────────────────┬───────────────┬──────────────┐
│ cohort_name                                                                             ┆ last_3m_avg_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                                                                     ┆               --- ┆           --- ┆          --- │
│ str                                                                                     ┆               f64 ┆           f64 ┆          f64 │
╞═════════════════════════════════════════════════════════════════════════════════════════╪═══════════════════╪═══════════════╪══════════════╡
│ Diseases of the circulatory system                                                      ┆          45859.94 ┆      19588.40 ┆       134.12 │
│ Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified ┆       

In [41]:
from cohorts_procedures import get_cohort_procedures

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Procedures: {compute_emerging_cohorts(get_cohort_procedures('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Procedures: {compute_emerging_cohorts(get_cohort_procedures('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Procedures: shape: (10, 4)
┌────────────────────────────────────────────────────────┬───────────────────┬───────────────┬──────────────┐
│ cohort_name                                            ┆ last_3m_avg_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                                    ┆               --- ┆           --- ┆          --- │
│ str                                                    ┆               f64 ┆           f64 ┆          f64 │
╞════════════════════════════════════════════════════════╪═══════════════════╪═══════════════╪══════════════╡
│ Medical Care | Whole Blood                             ┆          62632.25 ┆       8446.67 ┆       641.50 │
│ Lump Sum Purchase of DME, Prosthetics, Orthotics       ┆           6926.35 ┆       3998.65 ┆        73.22 │
│ Physical Therapy | Medical Care | Occupational Therapy ┆            491.79 ┆        323.50 ┆        52.02 │
│ Rental of DME                                          ┆            350.66 ┆        

In [42]:
from cohorts_facilities import get_cohort_facilities
pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_facilities('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_facilities('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Facilities: shape: (10, 4)
┌────────────────────────────────────────────────────┬───────────────────┬───────────────┬──────────────┐
│ cohort_name                                        ┆ last_3m_avg_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                                ┆               --- ┆           --- ┆          --- │
│ str                                                ┆               f64 ┆           f64 ┆          f64 │
╞════════════════════════════════════════════════════╪═══════════════════╪═══════════════╪══════════════╡
│ Walk-in Retail Health Clinic                       ┆             18.67 ┆          2.33 ┆       700.00 │
│ Non-residential Substance Abuse Treatment Facility ┆            162.64 ┆         20.33 ┆       700.00 │
│ Unassigned                                         ┆           1447.76 ┆        740.16 ┆        95.60 │
│ Urgent Care Facility                               ┆           1099.78 ┆        717.88 ┆        53.20 │
│ Rural Heal

In [43]:
from cohorts_drugs import get_cohort_drugs_usage
pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_drugs_usage('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_drugs_usage('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Facilities: shape: (10, 4)
┌─────────────────────────────────────────────────────────┬───────────────────┬───────────────┬──────────────┐
│ cohort_name                                             ┆ last_3m_avg_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                                     ┆               --- ┆           --- ┆          --- │
│ str                                                     ┆               f64 ┆           f64 ┆          f64 │
╞═════════════════════════════════════════════════════════╪═══════════════════╪═══════════════╪══════════════╡
│ Other Nervous System DrugsNasal Preparations            ┆             22.87 ┆          4.51 ┆       407.42 │
│ Pituitary And Hypothalamic Hormones And Analogues       ┆           1436.62 ┆        406.45 ┆       253.45 │
│ Other Nervous System Drugs                              ┆           8143.41 ┆       2361.95 ┆       244.78 │
│ Antiobesity Preparations, Excl. Diet Products           ┆             11.33 

In [44]:
from cohorts_demographics import get_cohort_demographics_ages, get_cohort_demographics_relationships, get_cohort_demographics_genders

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_demographics_ages('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_demographics_ages('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_demographics_genders('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_demographics_genders('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_demographics_relationships('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_demographics_relationships('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Facilities: shape: (5, 4)
┌─────────────┬───────────────────┬───────────────┬──────────────┐
│ cohort_name ┆ last_3m_avg_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---         ┆               --- ┆           --- ┆          --- │
│ str         ┆               f64 ┆           f64 ┆          f64 │
╞═════════════╪═══════════════════╪═══════════════╪══════════════╡
│ 19-26       ┆           7915.97 ┆       7978.74 ┆        -0.79 │
│ 41-64       ┆         143158.16 ┆     155195.28 ┆        -7.76 │
│ 27-40       ┆          26252.24 ┆      33986.09 ┆       -22.76 │
│ 0-18        ┆          13434.14 ┆      18078.07 ┆       -25.69 │
│ 65+         ┆           3908.44 ┆       9620.86 ┆       -59.38 │
└─────────────┴───────────────────┴───────────────┴──────────────┘
Cohort Facilities: shape: (2, 4)
┌─────────────┬───────────────────┬───────────────┬──────────────┐
│ cohort_name ┆ last_3m_avg_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---         ┆               --- ┆           --- ┆          --

In [45]:
from cohorts_providers import get_cohort_individual_medical_provider, get_cohort_medical_provider_speciality, get_cohort_individual_rx_provider

pl.Config.set_tbl_width_chars(8000) 
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
with pl.Config(fmt_str_lengths=200, set_fmt_float="full", float_precision=2):

    # print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_individual_medical_provider('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_individual_medical_provider('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_individual_rx_provider('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_individual_rx_provider('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")
    # print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_medical_provider_speciality('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=True).collect()}")
    print(f"Cohort Facilities: {compute_emerging_cohorts(get_cohort_medical_provider_speciality('113463').collect(), '2025-04-30', number_of_rows=10, include_infinity_percentage=False).collect()}")

Cohort Facilities: shape: (10, 4)
┌──────────────────────────────────────────────┬───────────────────┬───────────────┬──────────────┐
│ cohort_name                                  ┆ last_3m_avg_spend ┆ avg_24m_spend ┆ pct_increase │
│ ---                                          ┆               --- ┆           --- ┆          --- │
│ str                                          ┆               f64 ┆           f64 ┆          f64 │
╞══════════════════════════════════════════════╪═══════════════════╪═══════════════╪══════════════╡
│ US59 EMERGENCY CENTER  LLC                   ┆           1510.60 ┆        188.83 ┆       700.00 │
│ JORDAN DEHLI                                 ┆            323.00 ┆         40.37 ┆       700.00 │
│ JERROD FELDMAN                               ┆            311.20 ┆         38.90 ┆       700.00 │
│ NORTHEASTERN PENNSYLVANIA HEALTH CORPORATION ┆            254.11 ┆         31.76 ┆       700.00 │
│ DUSTIN VOSS                                  ┆            246.88